# Created by Jacek Grzybowski
## second attempt to make more accurate classification

## Part 1

In [1]:
# import libraries
import cv2
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Activation, MaxPooling2D, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

In [2]:
def load_img(indir, exclude = []):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if not os.path.isdir(indir+'/'+class_dir): 
            continue
        the_class = class_dir
        if the_class in exclude:
            continue
        print("Loading:",class_dir)
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('weather', ["rain", "shine"])
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels

Loading: sunrise
Loading: cloudy
loaded 657  samples
classes {'sunrise', 'cloudy'}


In [3]:
# create end train/test dataset

labels_vec = LabelEncoder().fit_transform(org_labels)
one_hot_labels = keras.utils.to_categorical(labels_vec)
# org_samples, one_hot_labels = shuffle(org_samples, one_hot_labels)

X_train, X_test, y_train, y_test = train_test_split(org_samples, one_hot_labels, test_size=0.25, random_state=666)

print(X_train.shape)
print(y_train.shape)


(492, 64, 64, 3)
(492, 2)


In [12]:
# create and compie model
model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
# train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=128, batch_size=16)

Epoch 1/128
31/31 [==============================] - 0s 6ms/step - loss: 0.4825 - accuracy: 0.8862 - val_loss: 2.5708 - val_accuracy: 0.7152
Epoch 2/128
31/31 [==============================] - 0s 4ms/step - loss: 0.1501 - accuracy: 0.9593 - val_loss: 0.8725 - val_accuracy: 0.8303
Epoch 3/128
31/31 [==============================] - 0s 4ms/step - loss: 0.0952 - accuracy: 0.9797 - val_loss: 0.1080 - val_accuracy: 0.9758
Epoch 4/128
31/31 [==============================] - 0s 3ms/step - loss: 0.0508 - accuracy: 0.9817 - val_loss: 0.0907 - val_accuracy: 0.9697
Epoch 5/128
31/31 [==============================] - 0s 3ms/step - loss: 0.0546 - accuracy: 0.9858 - val_loss: 0.4364 - val_accuracy: 0.8788
Epoch 6/128
31/31 [==============================] - 0s 4ms/step - loss: 0.0329 - accuracy: 0.9898 - val_loss: 0.0712 - val_accuracy: 0.9758
Epoch 7/128
31/31 [==============================] - 0s 3ms/step - loss: 0.0296 - accuracy: 0.9919 - val_loss: 0.2605 - val_accuracy: 0.9152
Epoch 8/128
3

In [14]:
# results
testResults = model.predict(X_test)

print(confusion_matrix(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print("Cohen's Kappa: {}".format(cohen_kappa_score(y_test.argmax(axis=1), testResults.argmax(axis=1))))
print("Accuracy: ",accuracy_score(y_test.argmax(axis=1), testResults.argmax(axis=1)))

[[75  0]
 [ 0 90]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        75
           1       1.00      1.00      1.00        90

    accuracy                           1.00       165
   macro avg       1.00      1.00      1.00       165
weighted avg       1.00      1.00      1.00       165

Cohen's Kappa: 1.0
Accuracy:  1.0


```
[[75  0]
 [ 0 90]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        75
           1       1.00      1.00      1.00        90

    accuracy                           1.00       165
   macro avg       1.00      1.00      1.00       165
weighted avg       1.00      1.00      1.00       165

Cohen's Kappa: 1.0
Accuracy:  1.0
```

## Part 2

In [17]:
def load_img(indir, exclude = []):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if not os.path.isdir(indir+'/'+class_dir): 
            continue
        the_class = class_dir
        if the_class in exclude:
            continue
        print("Loading:",class_dir)
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('weather')
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels



Loading: rain
Loading: sunrise
Loading: shine
Loading: cloudy
loaded 1123  samples
classes {'rain', 'shine', 'cloudy', 'sunrise'}


In [18]:
# create end train/test dataset

labels_vec = LabelEncoder().fit_transform(org_labels)
one_hot_labels = keras.utils.to_categorical(labels_vec)
# org_samples, one_hot_labels = shuffle(org_samples, one_hot_labels)

X_train, X_test, y_train, y_test = train_test_split(org_samples, one_hot_labels, test_size=0.25, random_state=4244)

print(X_train.shape)
print(y_train.shape)

(842, 64, 64, 3)
(842, 4)


In [24]:
# create model
model2 = Sequential()
model2.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model2.add(BatchNormalization())
model2.add(Activation("relu"))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(256))
model2.add(Activation("relu"))
model2.add(Dense(4))
model2.add(Activation("softmax"))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [27]:
# train model
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=256, batch_size=16)

Epoch 1/256
53/53 [==============================] - 0s 4ms/step - loss: 0.0112 - accuracy: 0.9952 - val_loss: 0.3518 - val_accuracy: 0.9324
Epoch 2/256
53/53 [==============================] - 0s 3ms/step - loss: 0.0077 - accuracy: 0.9976 - val_loss: 0.4277 - val_accuracy: 0.9253
Epoch 3/256
53/53 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 0.9988 - val_loss: 0.3968 - val_accuracy: 0.9110
Epoch 4/256
53/53 [==============================] - 0s 4ms/step - loss: 0.0038 - accuracy: 0.9988 - val_loss: 0.3954 - val_accuracy: 0.9004
Epoch 5/256
53/53 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 0.9988 - val_loss: 0.3412 - val_accuracy: 0.9324
Epoch 6/256
53/53 [==============================] - 0s 3ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.3818 - val_accuracy: 0.9110
Epoch 7/256
53/53 [==============================] - 0s 3ms/step - loss: 0.0086 - accuracy: 0.9964 - val_loss: 1.4582 - val_accuracy: 0.7402
Epoch 8/256
5

In [28]:
testResults = model2.predict(X_test)

print(confusion_matrix(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), testResults.argmax(axis=1)))
print("Cohen's Kappa: {}".format(cohen_kappa_score(y_test.argmax(axis=1), testResults.argmax(axis=1))))
print("Accuracy: ",accuracy_score(y_test.argmax(axis=1), testResults.argmax(axis=1)))

[[70  2  7  0]
 [ 2 60  2  0]
 [ 1  2 56  0]
 [ 0  0  0 79]]
              precision    recall  f1-score   support

           0       0.96      0.89      0.92        79
           1       0.94      0.94      0.94        64
           2       0.86      0.95      0.90        59
           3       1.00      1.00      1.00        79

    accuracy                           0.94       281
   macro avg       0.94      0.94      0.94       281
weighted avg       0.95      0.94      0.94       281

Cohen's Kappa: 0.9238250144014096
Accuracy:  0.9430604982206405


```
[[70  2  7  0]
 [ 2 60  2  0]
 [ 1  2 56  0]
 [ 0  0  0 79]]
              precision    recall  f1-score   support

           0       0.96      0.89      0.92        79
           1       0.94      0.94      0.94        64
           2       0.86      0.95      0.90        59
           3       1.00      1.00      1.00        79

    accuracy                           0.94       281
   macro avg       0.94      0.94      0.94       281
weighted avg       0.95      0.94      0.94       281

Cohen's Kappa: 0.9238250144014096
Accuracy:  0.9430604982206405
```